# 03 — Feedback Effects
Simulate auditor feedback from train, re‑score, and compare metrics.

In [ ]:
import pandas as pd

# Load round feedback data and add auditor_id column
feedback_df = pd.read_csv('../data/raw/round_feedback_full.csv')


,artifact_id,text,evidence_type,timestamp,gold_controls,partition,gold_rationale,auditor_id
0,2000,Multiple failed logins from 10.0.5.12 to auth-...,log,2025-09-15T09:00:00Z,AC-7;AU-6,feedback,Lockout threshold not enforced; correlated via...,1
1,2001,S3 bucket finance-archive not encrypted at res...,config,2025-09-15T09:03:00Z,SC-28;SC-12,feedback,At-rest encryption and key management gaps ⇒ S...,2
2,2002,NTP drift 62s on db-01; not synced to authorit...,log,2025-09-15T09:06:00Z,AU-8,feedback,Clock drift jeopardizes timestamp integrity ⇒ ...,3
3,2003,API gateway rate limiting disabled; prior DDoS...,log,2025-09-15T09:09:00Z,SC-5;SC-7,feedback,DoS protection missing; boundary controls impl...,4
4,2004,TLS disabled on api-gateway; HTTP allowed; wea...,config,2025-09-15T09:12:00Z,SC-8;SC-13,feedback,Insecure transport and crypto configuration ⇒ ...,5
